In [1]:
!pip install simpletransformers

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd

In [5]:
import os
import shutil

In [6]:
os.getcwd()

'C:\\Users\\Jayalakshmi'

In [7]:
from simpletransformers.classification import ClassificationModel


# Create a TransformerModel
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, args={'reprocess_input_data': True, 
                                                                           'overwrite_output_dir': True},use_cuda=False)

C:\Users\Jayalakshmi\anaconda3\envs\vnenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificatio

In [8]:
df=pd.read_csv(r"C:\Users\Jayalakshmi\Documents\CSV files\Tweets.csv")
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24-02-2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,24-02-2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)


In [9]:
df.shape

(14640, 15)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  float64
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [11]:
df.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [12]:
df.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [13]:
df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [14]:
df1 = df[['text', 'airline_sentiment']].copy()
df1.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [15]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df1, test_size = 0.2)

In [16]:
def making_label(st):
    if(st=='positive'):
        return 2
    elif(st=='neutral'):
        return 1
    else:
        return 0
    
train['label'] = train['airline_sentiment'].apply(making_label)
test['label'] = test['airline_sentiment'].apply(making_label)

print(train.shape)

(11712, 3)


In [17]:
train_df = pd.DataFrame({
    'text': train['text'][:1500].replace(r'\n', ' ', regex=True),
    'label': train['label'][:1500]
})

In [18]:
train_df['label'].value_counts()

0    941
1    313
2    246
Name: label, dtype: int64

In [19]:
test_df = pd.DataFrame({
    'text': test['text'][-400:].replace(r'\n', ' ', regex=True),
    'label': test['label'][-400:]
})

In [20]:
model.train_model(train_df)

C:\Users\Jayalakshmi\anaconda3\envs\vnenv\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Epochs 0/1. Running Loss:    0.4981:  47%|████▋     | 89/188 [23:12<23:29, 14.23s/it]


Epochs 0/1. Running Loss:    0.5910:  96%|█████████▌| 180/188 [43:28<01:48, 13.51s/it]


Epoch 1 of 1: 100%|██████████| 1/1 [45:17<00:00, 2717.31s/it]


(188, 0.7317829230364333)

In [21]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

C:\Users\Jayalakshmi\anaconda3\envs\vnenv\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|██████████| 50/50 [03:54<00:00,  4.70s/it]


In [22]:
import pickle
with open('SA_Classification_TrModel.pickle','wb') as f:
    pickle.dump(model, f)

In [23]:
result

{'mcc': 0.6461258954890183, 'eval_loss': 0.49664610758423805}

In [24]:
model_outputs

array([[ 1.61819851, -1.44266641, -2.01235652],
       [ 1.57498658, -1.44870746, -1.99644887],
       [ 0.73586732, -0.23604396, -1.51024556],
       ...,
       [-0.14678699,  0.20109005, -0.38494021],
       [-0.34479982,  0.38552687, -0.26930749],
       [-0.25942263,  0.45786735,  0.31352061]])

In [25]:
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))

true = test_df['label'].tolist()
predicted = lst    

In [26]:
import sklearn
mat = sklearn.metrics.confusion_matrix(true , predicted)
mat

array([[241,  12,   5],
       [ 21,  34,   9],
       [  8,  17,  53]], dtype=int64)

In [27]:
sklearn.metrics.classification_report(true,predicted,target_names=['positive','neutral','negative'])

'              precision    recall  f1-score   support\n\n    positive       0.89      0.93      0.91       258\n     neutral       0.54      0.53      0.54        64\n    negative       0.79      0.68      0.73        78\n\n    accuracy                           0.82       400\n   macro avg       0.74      0.71      0.73       400\nweighted avg       0.82      0.82      0.82       400\n'

In [28]:
sklearn.metrics.accuracy_score(true,predicted)

0.82

In [29]:
def get_result(statement):
    pickled_model = pickle.load(open('SA_Classification_TrModel.pickle', 'rb'))
    result = pickled_model.predict([statement])
    pos = np.where(result[1][0] == np.amax(result[1][0]))
    pos = int(pos[0])
    sentiment_dict = {2:'positive',0:'negative',1:'neutral'}
    print(sentiment_dict[pos])
    return sentiment_dict[pos]

In [30]:
get_result("Seat was not comfortable")

100%|██████████| 1/1 [00:00<00:00,  2.33it/s]

negative


'negative'

In [31]:
get_result("food was excellent")

100%|██████████| 1/1 [00:00<00:00,  1.36it/s]

neutral


'neutral'

In [32]:
get_result("where is the flight rn?")

100%|██████████| 1/1 [00:00<00:00,  1.25it/s]

neutral


'neutral'

In [33]:
df_new1 = df1[['text']].copy()
df_new1.head()

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...


In [34]:
df_new2 = df_new1[:1000]
df_new2

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...
...,...
995,@united by the time I finally get to Dallas I ...
996,@united I'm trying to get to my final destinat...
997,@united that guy really has no customer servic...
998,@united he has no priority and Iove it


In [39]:
def get_result_df(df: pd.DataFrame, text: list[str]):
       #get result dataframe 
    df['sentiment'] = df['text'].apply(get_result)
    return df

In [40]:
df_new3 = get_result_df(df_new2, ['text'])
df_new3.head()

100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


neutral


100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


negative


100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


...

negative


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


neutral


C:\Users\Jayalakshmi\AppData\Local\Temp\ipykernel_127300\3708838259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['text'].apply(get_result)


,text,sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,negative
2,@VirginAmerica I didn't today... Must mean I n...,negative
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [49]:
df_new3['text'].loc[df_new3.index[4]]

"@VirginAmerica and it's a really big bad thing about it"

In [48]:
df_new3['text'].loc[df_new3.index[3]]

'@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse'

In [50]:
df_new3['sentiment'].value_counts()

negative    586
neutral     229
positive    185
Name: sentiment, dtype: int64